In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [71]:
retail=pd.read_excel('Megastore Dataset.xlsx')
retail.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,33712,CA-2013-PP18955140-41461,2013-07-06,2013-07-11,Standard Class,PP-189551406,Paul Prost,Home Office,1040.0,Holyoke,...,FUR-BO-3643,Furniture,Bookcases,Bush Mission Pointe Library,301.96,2,0.0,60.392,28.98,Medium
1,33326,CA-2013-GP14740140-41604,2013-11-26,2013-11-30,Standard Class,GP-147401406,Guy Phonely,Corporate,1453.0,Leominster,...,OFF-ST-6292,Office Supplies,Storage,"Tennsco Lockers, Sand",104.90,5,0.0,25.176,8.07,High


<font size=3> Transaction Example </font>

In [72]:
retail.iloc[0]

Row ID                                  33712
Order ID             CA-2013-PP18955140-41461
Order Date                2013-07-06 00:00:00
Ship Date                 2013-07-11 00:00:00
Ship Mode                      Standard Class
Customer ID                      PP-189551406
Customer Name                      Paul Prost
Segment                           Home Office
Postal Code                            1040.0
City                                  Holyoke
State                           Massachusetts
Country                         United States
Region                             Eastern US
Market                                   USCA
Product ID                        FUR-BO-3643
Category                            Furniture
Sub-Category                        Bookcases
Product Name      Bush Mission Pointe Library
Sales                                  301.96
Quantity                                    2
Discount                                  0.0
Profit                            

<font size='4'> Data Preparation </font>

In order to generate product associations, the data must first be prepared for use in machine learning models.


<font size=3> Categorical Variable Selection </font>

In [73]:
retail.City.value_counts().head(5)

New York City    915
Los Angeles      747
Philadelphia     537
San Francisco    510
Santo Domingo    443
Name: City, dtype: int64

In [74]:
retail['Ship Mode'].value_counts()

Standard Class    30775
Second Class      10309
First Class        7505
Same Day           2701
Name: Ship Mode, dtype: int64

In [75]:
retail['Order Priority'].value_counts()

Medium      29433
High        15501
Critical     3932
Low          2424
Name: Order Priority, dtype: int64

In [76]:
retail.Region.value_counts().head(5)

Western Europe       5883
Central America      5616
Oceania              3487
Western US           3203
Southeastern Asia    3129
Name: Region, dtype: int64

In [77]:
retail.Category.value_counts()

Office Supplies    31289
Technology         10141
Furniture           9860
Name: Category, dtype: int64

In [78]:
retail.City.value_counts().head(5)

New York City    915
Los Angeles      747
Philadelphia     537
San Francisco    510
Santo Domingo    443
Name: City, dtype: int64

<font size=4> Encoding </font>

In [79]:
#Ship Mode - Ordinal Encoding
encoder=OrdinalEncoder()
retail['Ship Mode Encoded'] = encoder.fit_transform(retail[['Ship Mode']])
retail['Ship Mode Encoded'].value_counts()

3.0    30775
2.0    10309
0.0     7505
1.0     2701
Name: Ship Mode Encoded, dtype: int64

In [80]:
#Order Priority - Ordinal Encoding Manually
map = {'Low': 0, 'Medium': 1, 'High': 2, 'Critical': 3}
retail['Order Priority Encoded'] = retail ['Order Priority'].map(map)
retail['Order Priority Encoded'].value_counts()

1    29433
2    15501
3     3932
0     2424
Name: Order Priority Encoded, dtype: int64

In [81]:
#Region - One Hot Encoding 
retail=pd.get_dummies(retail, columns=['Region'])

In [82]:
#Category - One Hot Encoding
retail=pd.get_dummies(retail, columns=['Category'])

In [83]:
#Updated Retail Data Set
retail.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Region_Southern Asia,Region_Southern Europe,Region_Southern US,Region_Western Africa,Region_Western Asia,Region_Western Europe,Region_Western US,Category_Furniture,Category_Office Supplies,Category_Technology
0,33712,CA-2013-PP18955140-41461,2013-07-06,2013-07-11,Standard Class,PP-189551406,Paul Prost,Home Office,1040.0,Holyoke,...,0,0,0,0,0,0,0,1,0,0
1,33326,CA-2013-GP14740140-41604,2013-11-26,2013-11-30,Standard Class,GP-147401406,Guy Phonely,Corporate,1453.0,Leominster,...,0,0,0,0,0,0,0,0,1,0
2,33327,CA-2013-GP14740140-41604,2013-11-26,2013-11-30,Standard Class,GP-147401406,Guy Phonely,Corporate,1453.0,Leominster,...,0,0,0,0,0,0,0,0,1,0
3,33325,CA-2013-GP14740140-41604,2013-11-26,2013-11-30,Standard Class,GP-147401406,Guy Phonely,Corporate,1453.0,Leominster,...,0,0,0,0,0,0,0,0,1,0
4,33328,CA-2013-GP14740140-41604,2013-11-26,2013-11-30,Standard Class,GP-147401406,Guy Phonely,Corporate,1453.0,Leominster,...,0,0,0,0,0,0,0,0,1,0


<font size=4> Transactionalize the Data </font>

In [84]:
#Grouped each transaction by Order ID, so each product from a given transaction is grouped together. 
#The number of rows is the number of unique transactions.
z=retail.groupby('Order ID')['Product Name'].apply(list)
frame=pd.DataFrame(z)
frame

,Product Name
Order ID,
AE-2012-PO8865138-41184,"[Fellowes File Cart, Industrial, Epson Calcula..."
AE-2014-EB4110138-41926,"[Bush Stackable Bookrack, Pine, Accos Paper Cl..."
AE-2014-MY7380138-42004,"[Tenex Folders, Blue, Stiletto Letter Opener, ..."
AE-2015-GH4665138-42351,"[Motorola Headset, VoIP, Eldon Lockers, Blue, ..."
AE-2015-JD5790138-42070,"[Hon Color Coded Labels, Adjustable]"
...,...
ZA-2015-RC9960146-42257,"[Novimex Round Labels, Adjustable]"
ZA-2015-RP9390146-42099,"[Eaton Cards & Envelopes, Premium]"
ZA-2015-SM10005146-42241,"[Advantus Staples, Metal, Elite Trimmer, Steel]"


In [85]:
#One hot encode the transactions. Each Product gets its own column name.
te = TransactionEncoder()
te_ary = te.fit_transform(frame['Product Name'])
transformed=pd.DataFrame(te_ary, columns=te.columns_)
transformed

,"""While you Were Out"" Message Book, One Form per Page","#10 Gummed Flap White Envelopes, 100/Box",#10 Self-Seal White Envelopes,"#10 White Business Envelopes,4 1/8 x 9 1/2","#10- 4 1/8"" x 9 1/2"" Recycled Envelopes","#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes","#10-4 1/8"" x 9 1/2"" Premium Diagonal Seam Envelopes",#6 3/4 Gummed Flap White Envelopes,"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators","1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2"" X 11"" Cards, 25 Env./Pack",...,Zebra ZM400 Thermal Label Printer,Zebra Zazzle Fluorescent Highlighters,Zipper Ring Binder Pockets,i.Sound Portable Power - 8000 mAh,iHome FM Clock Radio with Lightning Dock,"iKross Bluetooth Portable Keyboard + Cell Phone Stand Holder + Brush for Apple iPhone 5S 5C 5, 4S 4",iOttie HLCRIO102 Car Mount,iOttie XL Car Mount,invisibleSHIELD by ZAGG Smudge-Free Screen Protector,netTALK DUO VoIP Telephone Service
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25723,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25724,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25725,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25726,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


<font size= 4> Cleaned Data Set </font>

In [86]:
#Round sales, profit, shipping cost

In [87]:
retail['Sales']=retail['Sales'].round(3)
retail['Profit']=retail['Profit'].round(3)
retail['Shipping Cost']=retail['Shipping Cost'].round(3)

In [88]:
#ensure int data type
retail['Ship Mode Encoded'] = retail['Ship Mode Encoded'].astype(int)

In [89]:
#Fill blank columns in postal code with null
retail['Postal Code'] = retail['Postal Code'].astype(str)
retail['Postal Code'].replace(' ', np.nan, inplace=True)

In [90]:
# Export cleaned retail data to CSV


#retail.to_csv('retail_task3.csv', index=False)
#exporting transactionalized data (transformed) to csv
#chunk_size = 1000 

#try:
    #for i in range(0, len(transformed), chunk_size):
       # transformed.iloc[i:i + chunk_size].to_csv('output.csv', mode='a', header=(i == 0), index=False)
#except Exception as e:
  #  print("Error during export:", e)

<font size=4> Apriori Algorithm </font>

The prepared transactional data will be used by the Apriori algorithm to identify products that are frequently purchased together.

In [91]:
transformed.head(2)

,"""While you Were Out"" Message Book, One Form per Page","#10 Gummed Flap White Envelopes, 100/Box",#10 Self-Seal White Envelopes,"#10 White Business Envelopes,4 1/8 x 9 1/2","#10- 4 1/8"" x 9 1/2"" Recycled Envelopes","#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes","#10-4 1/8"" x 9 1/2"" Premium Diagonal Seam Envelopes",#6 3/4 Gummed Flap White Envelopes,"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators","1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2"" X 11"" Cards, 25 Env./Pack",...,Zebra ZM400 Thermal Label Printer,Zebra Zazzle Fluorescent Highlighters,Zipper Ring Binder Pockets,i.Sound Portable Power - 8000 mAh,iHome FM Clock Radio with Lightning Dock,"iKross Bluetooth Portable Keyboard + Cell Phone Stand Holder + Brush for Apple iPhone 5S 5C 5, 4S 4",iOttie HLCRIO102 Car Mount,iOttie XL Car Mount,invisibleSHIELD by ZAGG Smudge-Free Screen Protector,netTALK DUO VoIP Telephone Service
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [92]:
#Generate Frequent Itemsets
sets = apriori(transformed, min_support=.0001, use_colnames=True, low_memory=True)
sets

,support,itemsets
0,0.000117,"(""While you Were Out"" Message Book, One Form p..."
1,0.000155,"(#10 Gummed Flap White Envelopes, 100/Box)"
2,0.000155,(#10 Self-Seal White Envelopes)
3,0.000272,"(#10 White Business Envelopes,4 1/8 x 9 1/2)"
4,0.000389,"(#10- 4 1/8"" x 9 1/2"" Recycled Envelopes)"
...,...,...
3537,0.000117,"(Rogers Lockers, Blue, Sanford Canvas, Blue)"
3538,0.000117,"(Satellite Sectional Post Binders, Staples)"
3539,0.000117,"(Stanley Pencil Sharpener, Water Color, Stockw..."
3540,0.000117,"(Xerox 1916, Staples)"


In [100]:
#Generate Association Rules
rules= association_rules(sets,metric='lift',min_threshold=.5)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(#10 White Business Envelopes,4 1/8 x 9 1/2)",(Staples),0.000272,0.008629,0.000117,0.428571,49.667954,0.000114,1.734900,0.980133
1,(Staples),"(#10 White Business Envelopes,4 1/8 x 9 1/2)",0.008629,0.000272,0.000117,0.013514,49.667954,0.000114,1.013423,0.988395
2,"(Acco Binding Machine, Recycled)","(Acco Binder, Clear)",0.001360,0.001749,0.000117,0.085714,49.005714,0.000114,1.091837,0.980929
3,"(Acco Binder, Clear)","(Acco Binding Machine, Recycled)",0.001749,0.001360,0.000117,0.066667,49.005714,0.000114,1.069971,0.981311
4,"(Tenex Trays, Blue)","(Acco Index Tab, Clear)",0.001477,0.002915,0.000117,0.078947,27.082105,0.000112,1.082549,0.964500


In [102]:
len(rules)

60

In [94]:
#Full association rules table 
#rules.to_csv('association_rules.csv', index=False)

<font size=4> Relevant Rules </font>

60 product associations were discovered. They will be evaluated based on confidence and lift.

In [95]:
rules_by_confidence=rules.sort_values(by='confidence', ascending=False)
rules_by_confidence.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
34,(Hon Olson Stacker Chairs),(Staples),0.000155,0.008629,0.000117,0.750000,86.918919,0.000115,3.965485,0.988649
36,(Hoover Shoulder Vac Commercial Portable Vacuum),(Staples),0.000233,0.008629,0.000117,0.500000,57.945946,0.000115,1.982743,0.982972
0,"(#10 White Business Envelopes,4 1/8 x 9 1/2)",(Staples),0.000272,0.008629,0.000117,0.428571,49.667954,0.000114,1.734900,0.980133
56,(Xerox 1916),(Staples),0.000272,0.008629,0.000117,0.428571,49.667954,0.000114,1.734900,0.980133
48,(Newell 34),(Staples),0.000272,0.008629,0.000117,0.428571,49.667954,0.000114,1.734900,0.980133


In [96]:
rules_by_lift=rules.sort_values(by='lift', ascending=False)
rules_by_lift.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
40,"(Memorex Flash Drive, USB)","(Jiffy Mailers, Set of 50)",0.000661,0.000544,0.000117,0.176471,324.302521,0.000116,1.213625,0.997576
41,"(Jiffy Mailers, Set of 50)","(Memorex Flash Drive, USB)",0.000544,0.000661,0.000117,0.214286,324.302521,0.000116,1.271886,0.997459
22,"(SanDisk Message Books, Recycled)","(Brother Fax and Copier, Color)",0.000544,0.000933,0.000117,0.214286,229.714286,0.000116,1.271540,0.996189
23,"(Brother Fax and Copier, Color)","(SanDisk Message Books, Recycled)",0.000933,0.000544,0.000117,0.125000,229.714286,0.000116,1.142235,0.996576
30,"(Fellowes Folders, Blue)","(Hamilton Beach Stove, Red)",0.001516,0.000428,0.000117,0.076923,179.916084,0.000116,1.082870,0.995952


In [97]:
rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
count,60.000000,60.000000,6.000000e+01,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.002208,0.002208,1.166045e-04,0.132326,80.890405,0.000114,1.206764,0.983504
std,0.002618,0.002618,1.093352e-19,0.135798,64.507488,0.000001,0.410860,0.010750
min,0.000155,0.000155,1.166045e-04,0.013514,19.315315,0.000111,1.012989,0.948891
25%,0.000700,0.000700,1.166045e-04,0.065957,42.880000,0.000114,1.069088,0.978448
50%,0.001458,0.001458,1.166045e-04,0.080014,58.209337,0.000115,1.084918,0.986553
75%,0.001769,0.001769,1.166045e-04,0.166667,98.953846,0.000115,1.191768,0.991373
max,0.008629,0.008629,1.166045e-04,0.750000,324.302521,0.000116,3.965485,0.997576


In [98]:
#Final Rules
rules1= association_rules(sets,metric='lift',min_threshold=85)
final_rules=rules1.sort_values(by=['confidence','lift'], ascending=[False,False])
final_rules.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
14,(Hon Olson Stacker Chairs),(Staples),0.000155,0.008629,0.000117,0.750000,86.918919,0.000115,3.965485,0.988649
11,"(Hamilton Beach Stove, Red)","(Fellowes Folders, Blue)",0.000428,0.001516,0.000117,0.272727,179.916084,0.000116,1.372916,0.994867
17,"(Jiffy Mailers, Set of 50)","(Memorex Flash Drive, USB)",0.000544,0.000661,0.000117,0.214286,324.302521,0.000116,1.271886,0.997459


<font size=4> Findings </font>

Customers who buy Hon Olson Stacker Chairs also buy Staples 75% of the time.

If a Hamilton Beach Stove is purchased, Fellowes Folders are also bought 27% of the time. 

When Jiffy Mailers are purchased, Memorex Flash Drives are also bought 21% of the time. With a lift of 324, this means the flash drive is 324 times more likely to be purchased when mailers are in the basket compared to random chance.

<font size=4> Significance </font>

To leverage the product associations, Allias Megastore should:
1. Improve online recommendations: Use the associations to suggest related products online. For example, customers purchasing a Hamilton Beach Stove should be recommended Fellowes Folders, increasing the chance of additional purchases.

2. Place Products Together: Display items with strong associations next to each other in physical stores to encourage cross-purchasing. For example, the Hon Olson Stacker Chairs and Staples should be placed side by side to promote joint sales.

3. Offer promotional Bundles: Create discounted bundles of frequently purchased-together items. For instance, the Hamilton Beach Stove and Fellowes Folders are bought together 27% of the time. Bundling them at a discount can boost sales of both products. 

4. Build Larger Baskets: Use strong product pairings (like Jiffy Mailers and Memorex Flash Drives) to promote larger baskets. Adding related products to bundles or cross-sell promotions can grow average transaction value and improve revenue per customer.